In [10]:
import pandas as pd
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import get_column_plot
from sdv.evaluation.single_table import evaluate_quality
import os

In [11]:
# Load the real data
real_data = pd.read_csv('../datasets/ben10_master.csv')

In [12]:
# Load or create metadata
metadata_path = 'metadata.json'

if os.path.exists(metadata_path):
    metadata = Metadata.load_from_json(metadata_path)
    print("Loaded metadata from metadata.json")
else:
    metadata = Metadata()
    metadata.detect_table_from_dataframe(
        table_name='ben10_table',
        data=real_data
    )
    metadata.save_to_json(metadata_path)
    print("Created and saved new metadata to metadata.json")

Loaded metadata from metadata.json


In [13]:
# Initialize and fit the CTGAN Synthesizer
synthesizer = CTGANSynthesizer(
    metadata,
    enforce_min_max_values = True,
    enforce_rounding=False,
    epochs=4000,
    verbose=True,
    cuda=True
)
synthesizer.fit(data=real_data)

Gen. (-0.57) | Discrim. (-0.52): 100%|██████████| 4000/4000 [06:33<00:00, 10.17it/s]  


In [14]:
# Generate synthetic data
synthetic_data = synthesizer.sample(num_rows=10000)

In [15]:
# Save the synthetic data to a CSV file
synthetic_data.to_csv('../datasets/ben10_CTGAN_synthetic.csv', index=False)
print("Synthetic dataset saved to ../datasets/ben10_CTGAN_synthetic.csv")

Synthetic dataset saved to ../datasets/ben10_synthetic.csv


In [16]:
# Evaluate the quality of the synthetic data
quality_report = evaluate_quality(
    real_data,
    synthetic_data,
    metadata
)
print("Column Shapes Details:", quality_report.get_details(property_name='Column Shapes'))
print("Column Pair Trends Details:", quality_report.get_details(property_name='Column Pair Trends'))

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 13/13 [00:00<00:00, 553.01it/s]|
Column Shapes Score: 93.81%

(2/2) Evaluating Column Pair Trends: |██████████| 78/78 [00:00<00:00, 186.14it/s]|
Column Pair Trends Score: 67.45%

Overall Score (Average): 80.63%

Column Shapes Details:                    Column        Metric     Score
0              alien_name  TVComplement  0.892800
1              enemy_name  TVComplement  0.930100
2          alien1_species  TVComplement  0.906800
3      alien1_home_planet  TVComplement  0.918800
4   alien1_strength_level  TVComplement  0.951267
5      alien1_speed_level  TVComplement  0.946433
6     alien1_intelligence  TVComplement  0.973300
7          alien2_species  TVComplement  0.926100
8      alien2_home_planet  TVComplement  0.936033
9   alien2_strength_level  TVComplement  0.981500
10     alien2_speed_level  TVComplement  0.958833
11    alien2_intelligence  TVComplement  0.950900
12                 winner  TVComplement  0.9223

In [17]:
fig = synthesizer.get_loss_values_plot()
fig.show()

In [18]:
# Visualize the distribution of a the winner column
fig = get_column_plot(
    real_data=real_data,
    synthetic_data=synthetic_data,
    column_name='winner',
    metadata=metadata
)

fig.show()